## Spotify Web API data


In [12]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import numpy as np
import matplotlib

In [13]:
from dotenv import load_dotenv
import os
import base64
from requests import post
import json

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
REDIRECT_URI = "https://localhost/"


def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}
token = get_token()

sp_oauth = SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri = REDIRECT_URI, scope = "user-library-read")

access_token = sp_oauth.get_access_token()

sp = spotipy.Spotify(auth_manager=sp_oauth)

C:\Users\justi\AppData\Local\Temp\ipykernel_41672\2948581706.py:36: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  access_token = sp_oauth.get_access_token()


In [72]:
playlist_ids = [
    '37i9dQZF1DWUZv12GM5cFk',
    '37i9dQZF1DX9Ol4tZWPH6V',
    '37i9dQZF1DX0P7PzzKwEKl',
    '37i9dQZF1DXaW8fzPh9b08',
    '37i9dQZF1DWTWdbR13PQYH',
    '37i9dQZF1DWWzQTBs5BHX9',
    '37i9dQZF1DX1vSJnMeoy3V',
    '37i9dQZF1DX3j9EYdzv2N9',
    '37i9dQZF1DWYuGZUE4XQXm',
    '37i9dQZF1DX4UkKv8ED8jp',
    '37i9dQZF1DXc6IFF23C9jj'
]
all_tracks = []
for id in playlist_ids:
    tracks = sp.playlist(id)['tracks']['items']
    for track in tracks:
        year = int(track['track']['album']['release_date'][:4])
        if year >= 2000 and year <= 2010:
            all_tracks.append(track['track'])

In [73]:
# ALL CATEGORIES FOR TRACKS
# album (get release date)
# artists
# available_markets (GET THE LENGTH)
# duration_ms (DIVIDE BY 1000)
# explicit
# name
# popularity

#MAKE SURE TO REMOVE DUPLICATES

211.16

In [120]:
df = pd.DataFrame(all_tracks)
df

,album,artists,available_markets,disc_number,duration_ms,episode,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,211160,False,False,{'isrc': 'USJI10000100'},{'spotify': 'https://open.spotify.com/track/6n...,https://api.spotify.com/v1/tracks/6naxalmIoLFW...,6naxalmIoLFWR0siv8dnQQ,False,Oops!...I Did It Again,80,https://p.scdn.co/mp3-preview/66a5cf0f69b1bb1e...,True,1,track,spotify:track:6naxalmIoLFWR0siv8dnQQ
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,224493,False,False,{'isrc': 'USIR20000145'},{'spotify': 'https://open.spotify.com/track/0v...,https://api.spotify.com/v1/tracks/0v1XpBHnsbkC...,0v1XpBHnsbkCn7iJ9Ucr1l,False,It's My Life,81,https://p.scdn.co/mp3-preview/71e6140994ab0db0...,True,1,track,spotify:track:0v1XpBHnsbkCn7iJ9Ucr1l
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, BO, BR, CA, CL, CO, CR, DO, EC, SV, GT, H...",1,200400,False,False,{'isrc': 'USJI10000001'},{'spotify': 'https://open.spotify.com/track/62...,https://api.spotify.com/v1/tracks/62bOmKYxYg7d...,62bOmKYxYg7dhrC6gH9vFn,False,Bye Bye Bye,74,https://p.scdn.co/mp3-preview/161d6b8cfeb94609...,True,1,track,spotify:track:62bOmKYxYg7dhrC6gH9vFn
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,284200,False,True,{'isrc': 'USIR10000448'},{'spotify': 'https://open.spotify.com/track/3y...,https://api.spotify.com/v1/tracks/3yfqSUWxFvZE...,3yfqSUWxFvZELEM4PmlwIR,False,The Real Slim Shady,87,https://p.scdn.co/mp3-preview/a3cb1372a99a0944...,True,8,track,spotify:track:3yfqSUWxFvZELEM4PmlwIR
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, BE, BO, BR, BG, CA, CL, CO, CR, CY, C...",1,307153,False,False,{'isrc': 'FR9W10100535'},{'spotify': 'https://open.spotify.com/track/49...,https://api.spotify.com/v1/tracks/49X0LAl6faAu...,49X0LAl6faAusYq02PRAY6,False,Lady - Hear Me Tonight,78,https://p.scdn.co/mp3-preview/05e58f0bcf5a9a29...,True,3,track,spotify:track:49X0LAl6faAusYq02PRAY6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,226533,False,False,{'isrc': 'USUM71002550'},{'spotify': 'https://open.spotify.com/track/1T...,https://api.spotify.com/v1/tracks/1T8PwF3JhIiW...,1T8PwF3JhIiWwBhun23RNe,False,Impossible,68,https://p.scdn.co/mp3-preview/b99277b285f3bc86...,True,2,track,spotify:track:1T8PwF3JhIiWwBhun23RNe
1020,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[US],1,188000,False,False,{'isrc': 'FRZID1000210'},{'spotify': 'https://open.spotify.com/track/6F...,https://api.spotify.com/v1/tracks/6FbGlkHKGkYq...,6FbGlkHKGkYqpSCuBgurn3,False,Gettin' Over You (feat. Fergie & LMFAO),50,https://p.scdn.co/mp3-preview/30733f10b0b9a827...,True,2,track,spotify:track:6FbGlkHKGkYqpSCuBgurn3
1021,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, MX, US]",1,207600,False,False,{'isrc': 'GB7TP0900040'},{'spotify': 'https://open.spotify.com/track/5e...,https://api.spotify.com/v1/tracks/5e0dZqrrTaoj...,5e0dZqrrTaoj6AIL7VjnBM,False,Written in the Stars (feat. Eric Turner),46,https://p.scdn.co/mp3-preview/f77870bd929834a9...,True,8,track,spotify:track:5e0dZqrrTaoj6AIL7VjnBM
1022,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,207493